# 1神经网络

In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [2]:
data = loadmat('ex4data1.mat')
X = data['X']
y = data['y']
X.shape, y.shape

((5000, 400), (5000, 1))

由于在训练时要将y变为(5000,10)的数据，故需要使用one-hot，独热编码

In [3]:
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder(sparse=False)
y_hot = encode.fit_transform(y)

y[0], y_hot[0]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

## 前向传播和损失函数
cost = $J \left(\theta \right) = - \frac{1}{m} \left[\sum\limits_{i=1}^m \sum\limits_{k=1}^K y_k^{\left(i \right)} log\left( h_{\theta}\left(x^{\left(i \right)}\right)\right)_k+\left(1-y_k^{\left(i \right)}\right) log\left(1-\left(h_{\theta}\left(x^{\left(i \right)}\right)\right)_k \right)\right] + \frac{\lambda}{2m}\sum\limits_{l=1}^{L-1}\sum\limits_{i=1}^{S_l}\sum\limits_{j=1}^{S_{l+1}}\left(\theta_{ji}^\left(l \right) \right)^2$ 此为加上正则项的损失函数

In [4]:
def sigmoid(z):
    return 1. / (1 + np.exp(-z))

In [5]:
def forward(X, theta1, theta2):
    
    a0 = np.insert(X, 0, values=np.ones(X.shape[0]), axis=1)
    z1 = np.dot(a0, theta1.T)
    a1 = np.insert(sigmoid(z1), 0, values=np.ones(z1.shape[0]), axis=1)
    z2 = np.dot(a1, theta2.T)
    y_hat = sigmoid(z2)
    
    return a0, z1, a1, z2, y_hat

In [7]:
def cost(params, input_size, hidden_size, num_labels, X, y, labmda):
    # 由于使用了优化函数，所以theta1和theta2要合并并且变为一维数组
    
    m = X.shape[0]
    
    #params还原theta1和theta2
    theta1 = params[0:(input_size+1) * hidden_size].reshape(hidden_size, (input_size+1))
    theta2 = params[(input_size+1) * hidden_size:].reshape(num_labels, (hidden_size+1))
    
    #前向传播
    a0, z1, a1, z2, y_hat = forward(X, theta1, theta2)

    # y_hat.shape = (m, 10) 注：这里不好使用向量化
    J = 0
    for i in range(m):
    
        J += np.sum(np.multiply(-y[i, :], np.log(y_hat[i, :])) - np.multiply((1 - y[i, :]), np.log(1 - y_hat[i, :]))) #一维数组相同位置相乘直接用multiply，并且此种方式得到的是一个一维数组
    
    return J / m
        
        

### theta随机初始化

In [8]:
def initialize():
    input_size = 400
    hidden_size = 25
    num_labels = 10
    labmda = 1.0
    params = np.random.randn(hidden_size * (input_size+1) + num_labels * (hidden_size+1)) * 0.01        #随机初始化
#     params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

    return params, input_size, hidden_size, num_labels, labmda

In [9]:
params, input_size, hidden_size, num_labels, labmda = initialize()

In [10]:
cost(params, input_size, hidden_size, num_labels, X, y_hot, labmda) #输入的y为特征拓展后的y_hot

6.937138442774584

## 正则化代价函数

In [11]:
def regularized_cost(params, input_size, hidden_size, num_labels, X, y, labmda):
    m = X.shape[0]
    
    #params还原theta1和theta2
    theta1 = params[0:(input_size+1) * hidden_size].reshape(hidden_size, (input_size+1))
    theta2 = params[(input_size+1) * hidden_size:].reshape(num_labels, (hidden_size+1))
    
    #前向传播
    a0, z1, a1, z2, y_hat = forward(X, theta1, theta2)

    # y_hat.shape = (m, 10) 注：这里不好使用向量化
    J = 0
    for i in range(m):
    
        J += np.sum(np.multiply(-y[i, :], np.log(y_hat[i, :])) - np.multiply((1 - y[i, :]), np.log(1 - y_hat[i, :])))
    
    reg_1 = labmda / (2 * m) * np.sum(np.power(theta1[:, 1:], 2))
    reg_2 = labmda / (2 * m) * np.sum(np.power(theta2[:, 1:], 2)) #正则化不包括bias的那个theta
    J = J / m + reg_1 + reg_2
    
    return J

In [12]:
regularized_cost(params, input_size, hidden_size, num_labels, X, y_hot, labmda)

6.9372425282588095

## 反向传播

In [13]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [14]:
def backprop(params, input_size, hidden_size, num_labels, X, y, labmda):
    m = X.shape[0]
    #params还原theta1和theta2
    theta1 = params[0:(input_size+1) * hidden_size].reshape(hidden_size, (input_size+1))
    theta2 = params[(input_size+1) * hidden_size:].reshape(num_labels, (hidden_size+1))
    
    #前向传播
    a0, z1, a1, z2, y_hat = forward(X, theta1, theta2)
    
    #反向传播
    dz2 = y_hat - y  #(m, 10)                                         #m, 10
    dtheta2 = 1. / m * np.dot(dz2.T, a1)                              #10, 26
    dz1 = np.multiply(np.dot(dz2, theta2), np.multiply(a1, 1 - a1))   #m, 26
    dtheta1 = 1. / m * np.dot((dz1[:, 1:]).T, a0)                     #25, 401, dz1的第一列是bias的导数值，故舍去
    
    #正则化
    dtheta1[:, 1:] = dtheta1[:, 1:] + labmda / m * theta1[:, 1:]
    dtheta2[:, 1:] = dtheta2[:, 1:] + labmda / m * theta2[:, 1:]      #第j列不参与正则化
    
    grads = np.concatenate((np.ravel(dtheta1), np.ravel(dtheta2)))
    
    return grads

In [15]:
grads = backprop(params, input_size, hidden_size, num_labels, X, y_hot, labmda)
grads.shape

(10285,)

下面代码谨慎使用，我i5-4210的处理器算了好久

In [322]:
import scipy.optimize as opt
fmin = opt.fmin_tnc(func=regularized_cost, x0=params, fprime=backprop,
                    args=(input_size, hidden_size, num_labels, X, y_hot, labmda))

In [324]:
fmin[0].shape

(10285,)

还是看的出来拟合效果还是不错的，但是文档里面给出的准确率是95.3%不知道是不是优化算法不同的原因

In [329]:
params_2 = fmin[0]
theta1 = params_2[0:(input_size+1) * hidden_size].reshape(hidden_size, (input_size+1))
theta2 = params_2[(input_size+1) * hidden_size:].reshape(num_labels, (hidden_size+1))
_, _, _, _, predict = forward(X, theta1, theta2)
predict = predict.argmax(axis=1) + 1
predict = predict.reshape(predict.shape[0], 1)
predict = predict[predict == y]
print("Accuracy is " + str(len(predict) / len(y) * 100) + "%")

Accuracy is 99.7%


## Gradient Checking 不带正则化的

如果误差小于$10^{-9}$则说明执行正确

In [51]:
def gradient_checking(params, input_size, hidden_size, num_labels, X, y, epsilon=0.0001):
    m = X.shape[0]
    #params还原theta1和theta2
    theta1 = params[0:(input_size+1) * hidden_size].reshape(hidden_size, (input_size+1))
    theta2 = params[(input_size+1) * hidden_size:].reshape(num_labels, (hidden_size+1))
    
    #前向传播
    a0, z1, a1, z2, y_hat = forward(X, theta1, theta2)
    
    #反向传播
    dz2 = y_hat - y  #(m, 10)                                         #m, 10
    dtheta2 = 1. / m * np.dot(dz2.T, a1)                              #10, 26
#     dz1 = np.multiply(np.dot(dz2, theta2), np.multiply(a1, 1 - a1))   #m, 26
#     dtheta1 = 1. / m * np.dot((dz1[:, 1:]).T, a0)                     #25, 401, dz1的第一列是bias的导数值，故舍去
    
    #上面是backprop里面的直接抄过来
    #checking   此程序只检查theata2
    check = np.zeros(theta2.shape)
    for i in range(theta2.shape[0]):
        for j in range(theta2.shape[1]):
            paramsplus = params
            paramsmins = params
            paramsplus[i * theta2.shape[1] + j + theta1.shape[0] * theta1.shape[1]] += epsilon
            paramsmins[i * theta2.shape[1] + j + theta1.shape[0] * theta1.shape[1]] -= epsilon
            check[i, j] = (cost(paramsplus, input_size, hidden_size, num_labels, X, y, 1.0) - cost(paramsmins, input_size, hidden_size, num_labels, X, y, 1.0)) / (2 * epsilon)
#             print(cost(paramsplus, input_size, hidden_size, num_labels, X, y, 1.0), cost(paramsmins, input_size, hidden_size, num_labels, X, y, 1.0))
    
    return check, dtheta2

In [52]:
check, dtheta2 = gradient_checking(params, input_size, hidden_size, num_labels, X, y_hot)

下面是误差，感觉有点大了，但是还没检查出来是哪个步骤出错了,但是我将另外一个大神的代码改了改也拿来算梯度检测发现和我的误差是一样的，所以不知道是不是我梯度检测是方式错了。 后来发现epsilon小于100损失函数根本没有变化check全为0，所以有可能是上面计算check的地方错误了，待修正

In [54]:
np.sum(np.power(check - dtheta2, 2)) / (np.sum(np.power(check, 2)) + np.sum(np.power(dtheta2, 2))) #计算梯度检测的结果

1.0

## Visualizing the hidden layer未做